In [2]:
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import callbacks
from keras import backend as K
from keras.datasets import mnist
from matplotlib import pyplot as plt
from matplotlib import cm
from PIL import Image
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import jaccard_score,log_loss, mean_absolute_error,median_absolute_error
%matplotlib inline
from tqdm import tqdm
import pickle
def makefile(what,filename):
    with open(filename,"wb") as f3:
        pickle.dump(what,f3)

def readfile(filename):
    with open(filename,"rb") as f4:
        ans=pickle.load(f4)
    return ans

Using TensorFlow backend.


In [3]:
from keras.datasets import cifar10
from sklearn.metrics import roc_auc_score
from sklearn.metrics import balanced_accuracy_score
from keras.backend import clear_session

In [4]:
#The path of goal image. Please use your own goal image :)
images=["Lenna.png"]
#images=["image1.png","image2.png","image3.png"]

In [5]:
#dataset CIFAR10
#Please apply your own dataset :)
#train
train=cifar10.load_data()[0][0]
train_label=cifar10.load_data()[0][1]
train_label=train_label.reshape(len(train_label))
#test
x_test=cifar10.load_data()[1][0]
y_test=cifar10.load_data()[1][1]
y_test=y_test.reshape(len(y_test))
data3=np.zeros([len(train),32,32,3])
for n in range(len(train)):
    data3[n]=Image.fromarray(train[n].astype("uint8"))
train=data3

In [6]:
#image size
M=train.shape[1]
N=train.shape[2]
C=train.shape[3]

In [ ]:
degree=[0,90,180,270]
values=[0,0.05,0.1,0.15,0.2,0.25,0.3,0.35,0.4]
convo=[6]

for goal in images:
    #goal image I, equation(3)
    I=Image.open(goal).resize([M,N])
    
    AUC_list=[]
    BA_list=[]
    Error_list=[]
    # Q is corresponding to Fig.6.
    for Q in convo:
        AUC=[]
        BA=[]
        Error=[]
        # e is random_state. We use 5 different random seed.
        for e in range(1):
            ensemble=[]
            #Rotation
            for rot in degree:
                error_record=[]
                
                
                balanced_accuracy=np.zeros([10,9])
                AUC_score=np.zeros(10)
                for i in range(10):
                    clear_session()
                    x_train=train[train_label==i]
                    I_train=np.zeros([len(x_train),M,N,C])
                    for n in range(len(x_train)):
                        I_train[n]=I.rotate(rot)
                    I_train=I_train
                    
                    x_test=x_test.reshape(len(x_test),M,N,C)
                    y_label=np.zeros(len(y_test))
                    y_label[y_test==i]=1
                    # バックエンドに依存したチャネルの位置を調整する
                    if K.image_data_format() == 'channels_last':
                        x_train = x_train.reshape(x_train.shape[0],
                                                  M, N,C)
                        #x_test = x_test.reshape(x_test.shape[0],image_height, image_width, 1)
                        I_train = I_train.reshape(I_train.shape[0],
                                                  M, N,C)
                        input_shape = (M, N,C)
                    else:
                        x_train = x_train.reshape(x_train.shape[0],
                                                  C, M, N)
                        x_test = x_test.reshape(x_test.shape[0],C, M, N)
                        input_shape = (C, M, N)

                    x_train = x_train.astype('float32')
                    I_train = I_train.astype('float32')
                    x_train, x_valid, I_train, I_valid = train_test_split(x_train, I_train, test_size=0.3, random_state=e)
                    x_valid, x_threshold, I_valid, I_threshold = train_test_split(x_valid, I_valid, test_size=0.3, random_state=e)
                    #x_test = x_test.astype('float32')
                    #x_train = (x_train - x_train.min()) / (x_train.max() - x_train.min())
                    #x_test = (x_test - x_test.min()) / (x_test.max() - x_test.min())
                    
                    # model f:X → I
                    model = models.Sequential()
                    # Repeat convolution layer Q times
                    model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same',input_shape=input_shape))
                    for n in range(Q):
                        model.add(layers.Conv2D(32,kernel_size=(3,3),activation='relu',padding='same'))
                    model.add(layers.Conv2D(C, kernel_size=(3, 3),activation='relu', padding='same'))
                    # 28 x 28 x 1
                    model.compile(optimizer='adam',
                                  loss='mean_absolute_error')
                    fit_callbacks = [
                        callbacks.EarlyStopping(monitor='val_loss',
                                                patience=5,
                                                mode='min')
                    ]
                    # Train model
                    model.fit(x_train, I_train,
                              epochs=100,
                              batch_size=16,
                              shuffle=True,
                              validation_data=(x_valid, I_valid),callbacks=fit_callbacks)
                    #f_threshold = ITN output from x_threshold
                    f_threshold=model.predict(x_threshold)
                    f_test=model.predict(x_test)
                    I_train=I_train.reshape(I_train.shape[0],int(M*N) ,C)
                    goal_image=I_train[0]
                    f_threshold=f_threshold.reshape(f_threshold.shape[0],int(M*N),C)
                    f_test=f_test.reshape(f_test.shape[0],int(M*N),C)
                    # equation (12)
                    Eth=np.zeros(len(f_threshold))
                    
                    for n in range(len(f_threshold)):
                        Eth[n]=mean_absolute_error(goal_image,f_threshold[n])
                    Etest=np.zeros(len(f_test))
                    for n in range(len(f_test)):
                        Etest[n]=mean_absolute_error(goal_image,f_test[n])
                    error_record.append(Etest)
                    #y_one=np.ones(len(Eth))
                    #y_outlier=np.zeros(len(Etest))
                    #data=np.concatenate([Eth,Etest])
                    #label=np.concatenate([y_one,y_outlier])
                    #plt.scatter(data,label)
                    #plt.hist([Eth,Etest],range=(0,0.3),stacked=False,bins=10)
                    m=0
                    #multiply construction error by (-1)
                    AUC_score[i]=roc_auc_score(y_label,(-1)*Etest)
                    for p in values:
                        y_pred=np.zeros(len(y_label))
                        threshold=int(len(Eth)*p)+1
                        #print(Eth[Eth.argsort()[-threshold]])
                        y_pred[Etest<=Eth[Eth.argsort()[-threshold]]]=1
                        #Etest[Etest>Eth[Eth.argsort()[-threshold]]
                        balanced_accuracy[i][m]=balanced_accuracy_score(y_label,y_pred)
                        m+=1
                ensemble.append(error_record)
            AUC.append(AUC_score)
            BA.append(balanced_accuracy)
            Error.append(ensemble)
        AUC_list.append(AUC)
        BA_list.append(BA)
        Error_list.append(Error)
    #makefile(outputs,"result_ocitn/cifar10_"+str(goal)+".pkl")

In [40]:
np.array(AUC_list).mean(axis=0)

array([[0.74706756, 0.84600756, 0.679689  , 0.57218122, 0.70011206,
        0.70850172, 0.76164672, 0.7982695 , 0.81680889, 0.7790615 ]])

In [41]:
np.array(BA_list).mean(axis=0)

array([[[0.50033333, 0.53866667, 0.58238889, 0.6245    , 0.66372222,
         0.67672222, 0.68772222, 0.69472222, 0.69416667],
        [0.50061111, 0.57905556, 0.67672222, 0.73938889, 0.77705556,
         0.78844444, 0.78561111, 0.77166667, 0.75      ],
        [0.50177778, 0.54744444, 0.57244444, 0.59211111, 0.60672222,
         0.62022222, 0.6255    , 0.63422222, 0.63722222],
        [0.50194444, 0.52855556, 0.54172222, 0.5485    , 0.54844444,
         0.5555    , 0.54972222, 0.54955556, 0.54133333],
        [0.50605556, 0.56066667, 0.60433333, 0.61805556, 0.62905556,
         0.64005556, 0.64661111, 0.64611111, 0.65105556],
        [0.5005    , 0.55822222, 0.60883333, 0.62894444, 0.64527778,
         0.65094444, 0.65277778, 0.6515    , 0.64922222],
        [0.49977778, 0.58594444, 0.64511111, 0.67388889, 0.69433333,
         0.69611111, 0.70055556, 0.69455556, 0.68522222],
        [0.50194444, 0.58077778, 0.64488889, 0.67672222, 0.70727778,
         0.71922222, 0.72594444, 0.7266666

In [62]:
table=[]
for i in range(10):
    raw=[]
    y_label=np.zeros(len(y_test))
    y_label[y_test==i]=1
    raw.append(roc_auc_score(y_label,(-1)*ensemble[0][i]))
    raw.append(roc_auc_score(y_label,(-1)*ensemble[1][i]))
    raw.append(roc_auc_score(y_label,(-1)*ensemble[2][i]))
    raw.append(roc_auc_score(y_label,(-1)*ensemble[3][i]))
    raw.append(roc_auc_score(y_label,(-1)*np.array(ensemble).sum(axis=0)[i]))
    table.append(raw)

In [63]:
pd.DataFrame(table)

,0,1,2,3,4
0,0.758308,0.759212,0.768697,0.747068,0.787437
1,0.868262,0.852371,0.847851,0.846008,0.890255
2,0.702346,0.684322,0.681972,0.679689,0.718307
3,0.584173,0.573977,0.574705,0.572181,0.602823
4,0.713201,0.676324,0.686718,0.700112,0.723875
5,0.701446,0.652800,0.689796,0.708502,0.738677
6,0.774859,0.726189,0.733957,0.761647,0.810552
7,0.780821,0.784701,0.783208,0.798269,0.820728
8,0.831943,0.784953,0.841678,0.816809,0.853811
9,0.841771,0.816374,0.816872,0.779062,0.860756


In [61]:
np.array(ensemble).sum(axis=0)[0]

array([81.31238556, 79.18724442, 83.80313492, ..., 89.6935463 ,
       87.7099514 , 82.91192245])